# Filtrando itens já consumido mab2rec

In [7]:
import pandas as pd
from mab2rec import BanditRecommender, LearningPolicy
from mab2rec.pipeline import train
from sklearn.preprocessing import LabelEncoder
import implicit
from scipy.sparse import csr_matrix
from implicit.nearest_neighbours import bm25_weight
import plotly.express as px
import time
import os
import math

train_data = "../data/ml100k/data_train.csv"
test_data = "../data/ml100k/data_test.csv"

In [8]:
df_test = pd.DataFrame([
    {
        'user_id': 1,
        'item_id': 1,
        'response': 1
    },
    {
        'user_id': 2,
        'item_id': 2,
        'response': 1
    },
    {
        'user_id': 2,
        'item_id': 3,
        'response': 1
    },
    {
        'user_id': 2,
        'item_id': 4,
        'response': 1
    },
    {
        'user_id': 2,
        'item_id': 5,
        'response': 1
    },
    {
        'user_id': 3,
        'item_id': 2,
        'response': 1
    }
])

contexts = [[1, 0, 0], [0, 1, 0], [0, 0, 1]]

contexts_df = pd.DataFrame({
    'user_id': [1, 2, 3],
    'u0': [contexts[0][0], contexts[1][0], contexts[2][0]],
    'u1': [contexts[0][1], contexts[1][1], contexts[2][1]],
    'u2': [contexts[0][2], contexts[1][2], contexts[2][2]]
})
print(contexts_df)

   user_id  u0  u1  u2
0        1   1   0   0
1        2   0   1   0
2        3   0   0   1


In [9]:
rec_model = BanditRecommender(LearningPolicy.LinGreedy(0.1), top_k=1)
train(rec_model, data=df_test, user_features=contexts_df)

In [10]:
rec_model.recommend(contexts=[[1, 0, 0]])  # Retorna 1, que já foi consumido pelo usuário 1

[1]

In [11]:
rec_model.recommend(contexts=[[1, 0, 0]], excluded_arms=[[1]])

[5]

In [12]:
rec_model.recommend(contexts=[[1, 0, 0]], excluded_arms=[[1, 5]])

[4]

In [57]:
def group_interactions_by_user(interactions_df):
    interactions_by_user = interactions_df\
                        .groupby('user_id')[['item_id']]\
                        .apply(lambda df_user: df_user['item_id'].tolist())\
                        .reset_index(name='interactions')
    interactions_by_user = interactions_by_user.reset_index(drop=True)
    return interactions_by_user

In [60]:
grouped = group_interactions_by_user(df_test)

In [61]:
df_test.merge(grouped, on='user_id').drop(columns=['user_id', 'item_id', 'response']).values

array([[list([1])],
       [list([2, 3, 4, 5])],
       [list([2, 3, 4, 5])],
       [list([2, 3, 4, 5])],
       [list([2, 3, 4, 5])],
       [list([2])]], dtype=object)